# (CTC-17) Projeto de Buscas
---

## Dependências

In [210]:
import numpy as np
import random
from copy import copy, deepcopy

## Importando os dados

In [211]:

game = np.loadtxt("./games/game1.txt", dtype='str', delimiter=' ')
print(game)

[[&#39;-&#39; &#39;-&#39; &#39;1&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39;]
 [&#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;0&#39; &#39;X&#39; &#39;-&#39; &#39;-&#39;]
 [&#39;-&#39; &#39;0&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;2&#39;]
 [&#39;-&#39; &#39;0&#39; &#39;-&#39; &#39;X&#39; &#39;-&#39; &#39;X&#39; &#39;-&#39;]
 [&#39;X&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;1&#39; &#39;-&#39;]
 [&#39;-&#39; &#39;-&#39; &#39;3&#39; &#39;1&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39;]
 [&#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;-&#39; &#39;X&#39; &#39;-&#39; &#39;-&#39;]]


## Pré-Game

In [275]:
class Number():
    def __init__(self, pos, number):
        self.pos = pos
        self.n = number

class State():
    def __init__(self, game):
        self.state = copy(game)
        self.score = 0
        self.lamps = list()
        self.numbers = list()
        self.enemyLamps = 0
        self.darkSpots = 0

        ## Set lamps initial positions
        for j in range (7): # Colunas 
            freePositions = 0
            for i in range (7): # Linhas
                if self.state[i][j] == '-': 
                    freePositions += 1
                else:
                    if freePositions > 0:
                        l = random.randrange(i - freePositions, i)
                        self.lamps.append((l, j))
                        freePositions = 0
                    
                    ## If it is a number
                    if self.state[i][j] != '-' and self.state[i][j] != 'X':
                        self.numbers.append( Number( (i, j), int( self.state[i][j] ) ) )

            if freePositions > 0:
                l = random.randrange(7 - freePositions, 7)
                self.lamps.append((l, j))
        
        for l in self.lamps:
            self.state[l[0]][l[1]] = 'L'
     
        print("Estado Inicial:")
        print(self.state)

        enemy = 0
        for i in range (7):
            lamps = 0
            for j in range (7):
                ## lamp found
                if self.state[i][j] == 'L': 
                    if lamps != 0:
                        enemy += 1
                    lamps = 1
                ## wall found
                elif self.state[i][j] != 'L' and self.state[i][j] != '-':
                     lamps = 0
        
        self.enemyLamps = enemy
        self.score = self.enemyLamps
    
    def evaluateScore(self):
        enemyLamps = 0
        for i in range (7):
            lamps = 0
            for j in range (7):
                ## lamp found
                if self.state[i][j] == 'L': 
                    if lamps != 0:
                        enemyLamps += 1
                    lamps = 1
                ## wall found
                elif self.state[i][j] != 'L' and self.state[i][j] != '-':
                     lamps = 0
        
        print("Hey:")
        darkSpots = set()
        for i in range(7):
            for j in range(7):
                if self.state[i][j] == '-':
                    darkSpots.add((i, j))

        print("DarkSpots:")
        print(len(darkSpots))
        
        for lamp in self.lamps:
            # Search in column
            for i in range(lamp[0] + 1, 7):
                if self.state[i][lamp[1]] == '-':
                    if (i, lamp[1]) in darkSpots:
                        darkSpots.remove((i, lamp[1]))
                else:
                    break

            for i in range(lamp[0] - 1, -1, -1):
                if self.state[i][lamp[1]] == '-':
                    if (i, lamp[1]) in darkSpots:
                        darkSpots.remove((i, lamp[1]))
                else:
                    break
            
            # Search in line
            for j in range(lamp[1] + 1, 7):
                if self.state[lamp[0]][j] == '-':
                    if (lamp[0], j) in darkSpots:
                        darkSpots.remove((lamp[0], j))
                else:
                    break

            for j in range(lamp[1] - 1, -1, -1):
                if self.state[lamp[0]][j] == '-':
                    if (lamp[0], j) in darkSpots:
                        darkSpots.remove((lamp[0], j))
                else:
                    break
        
        print("DarkSpots:")
        print(darkSpots)


In [277]:
def generateNextState(current): 
    nextState = deepcopy(current)

    # Select a lamp
    l = random.choice(nextState.lamps)
    action = random.randrange(0, 2)

    ## retirar a lampada
    if action == 0:
        nextState.lamps.remove(l)
        nextState.state[l[0]][l[1]] = '-'
        nextState.evaluateScore()
    ## Mover a lampada
    else:
        validPositions = list()
        for i in range(l[0] + 1, 7):
            if nextState.state[i][l[1]] == '-':
                validPositions.append((i, l[1]))
            else:
                break

        for i in range(l[0] - 1, -1, -1):
            if nextState.state[i][l[1]] == '-':
                validPositions.append((i, l[1]))
            else:
                break

        if(len(validPositions) != 0):
            newLamp = random.choice(validPositions)
            nextState.lamps.remove(l)
            nextState.state[l[0]][l[1]] = '-'
            nextState.lamps.append(newLamp)
            nextState.state[newLamp[0]][newLamp[1]] = 'L'
            nextState.evaluateScore()

    return nextState

### Preenchendo a estrutura de dados

In [278]:
current = State(game)
nextState = generateNextState(current)
print(nextState.state)

Estado Inicial:
[[&#39;-&#39; &#39;L&#39; &#39;1&#39; &#39;L&#39; &#39;L&#39; &#39;-&#39; &#39;L&#39;]
 [&#39;L&#39; &#39;-&#39; &#39;-&#39; &#39;0&#39; &#39;X&#39; &#39;L&#39; &#39;-&#39;]
 [&#39;-&#39; &#39;0&#39; &#39;-&#39; &#39;L&#39; &#39;-&#39; &#39;-&#39; &#39;2&#39;]
 [&#39;-&#39; &#39;0&#39; &#39;L&#39; &#39;X&#39; &#39;-&#39; &#39;X&#39; &#39;L&#39;]
 [&#39;X&#39; &#39;-&#39; &#39;-&#39; &#39;L&#39; &#39;-&#39; &#39;1&#39; &#39;-&#39;]
 [&#39;-&#39; &#39;-&#39; &#39;3&#39; &#39;1&#39; &#39;L&#39; &#39;-&#39; &#39;-&#39;]
 [&#39;L&#39; &#39;L&#39; &#39;L&#39; &#39;L&#39; &#39;X&#39; &#39;L&#39; &#39;-&#39;]]
Hey:
DarkSpots:
20
2 0
3 0
0 0
1 1
1 2
5 0
1 1
0 0
5 1
4 1
2 4
2 5
2 2
4 4
0 5
4 4
3 4
2 4
5 5
5 6
2 5
0 5
1 6
5 5
6 6
1 6
0 5
4 6
5 6
6 6
3 2
2 2
1 2
4 1
DarkSpots:
set()
[[&#39;-&#39; &#39;L&#39; &#39;1&#39; &#39;L&#39; &#39;L&#39; &#39;-&#39; &#39;L&#39;]
 [&#39;L&#39; &#39;-&#39; &#39;-&#39; &#39;0&#39; &#39;X&#39; &#39;L&#39; &#39;-&#39;]
 [&#39;-&#39; &#39;0&#39; &#

In [ ]:
while True:
    current = State(game)
    plato = 20

    while True:
        nextState = generateNextState(current)
        if nextState.score == 0:
            break

        if nextState.score < current.score:
            current = nextState
            plato = 20
        else
            plato -= 1
            if plato == 0
                break

    if nextState.score == 0:
        current.print()
        break
